In [7]:
%%capture
!pip install geopandas
!pip install sddk

In [28]:
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import requests
from concurrent.futures import ThreadPoolExecutor
import time
import sddk

In [29]:
s = sddk.cloudSession("sciencedata.dk", "SDAM_root", "648597@au.dk")

Your ScienceData username (e.g. '123456@au.dk'): 648597@au.dk
Your ScienceData password: ··········
connection with shared folder established with you as its owner
endpoint variable has been configured to: https://sciencedata.dk/files/SDAM_root/


In [4]:
# import the datasets...
PHI = pd.read_json("https://zenodo.org/record/4972968/files/PHI_v1.0.json?download=1")

In [5]:
PHI_sample = PHI.sample(1000, random_state=1)
PHI_sample.head(5)

,URL,Book,Text,hdr1,hdr2,tildeinfo,note,lines,metadata,data,...,lem_sents,lemmata,raw_date,dating_dict,not_before,not_after,or,date_tags,certainty,link
60228,/text/180672?location=1691&patt=&bookid=225&of...,Epigr. tou Oropou,487,"Regions\nCentral Greece (IG VII-IX)\nMegaris, ...",Epigr. tou Oropou\n487,Boiotia — Oropos (Skala) — Amphiareion — 3rd c...,"Dedication. Small base of white marble, broke...",2,1\n,[— — —]εθνη[— — —]\n[— — —] ἀνέθηκε.,...,"[[ἔθνος, ἀνατίθημι]]","[ἔθνος, ἀνατίθημι]",3rd c. BC,"{'or': None, 'not_before': -300, 'not_after': ...",-300.0,-201.0,None,"[range, cent]",None,None
159554,/text/293424?location=1703&patt=&bookid=172&bo...,SEG,32:161,"Regions\nEgypt, Nubia and CyrenaïcaEgypt and N...",SEG\n32:161,Att. — Athens — 402/1 BC,,114,stele I.1\n\n\n\n5\n\ncol. I.6\n\n\n\n10\n\n\n...,Εὐθύμαχ[ο]ς ΑΛ̣Κ̣[— — — c.14 — — — ἐγραμμάτ]-\...,...,"[[εὐθυμάχος, γραμματεύω, αἰγηίδος, ἕβδομος, πρ...","[εὐθυμάχος, γραμματεύω, αἰγηίδος, ἕβδομος, πρυ...",402/1 BC,"{'or': None, 'not_before': -402, 'not_after': ...",-402.0,-401.0,None,[range],None,None
209067,/text/344689?location=1665&patt=&bookid=874&of...,"IG XII,6",2:944,"Regions\nAegean Islands, incl. Crete (IG XI-[X...","IG XII,6\n2:944",Aeg. Islands — Samos Isl. — Prov. unkn. [Samos...,"Epitaph. Plaque of gray limestone, intact at ...",7,\n\n\n\n\n5\n,"vacat 0,03\n[— — — χ]αῖρε\n[— — — —]α̣ς μνή-\n...",...,"[[χαίρω, μνήμη, μήτε, ξαϲ, ρου]]","[χαίρω, μνήμη, μήτε, ξαϲ, ρου]",5th/6th c. AD,"{'or': None, 'not_before': 401, 'not_after': 6...",401.0,600.0,None,"[range, cent, morece]",None,None
149118,/text/282987?location=1617&patt=&bookid=663&of...,IK Rhod. Peraia,220,"Regions\nAsia Minor\nCaria, Rhodian Peraia",IK Rhod. Peraia\n220,"Car., Rhod.Per. — Tymnos (Bozburun) — Hellenis...",,2,1\n,Θ̣αρσύν[ο]ντι\nἥρωι.,...,"[[θαρσύνοντι, ἥρως]]","[θαρσύνοντι, ἥρως]",Hellenistic period,"{'or': None, 'source': 'PeriodO', 'notes': Non...",-330.0,-30.0,None,"[range, period]",None,http://n2t.net/ark:/99152/p03wskd55sw
52953,/text/170921?location=1687&patt=&bookid=190&of...,IK Byzantion,74,Regions\nThrace and the Lower Danube (IG X)\nT...,IK Byzantion\n74,Thrace — Byzantion (Istanbul) — 2nd c. BC — Fı...,,2,1\n,Εὐκλέων\nΜενίσκου.,...,"[[εὐκλεής, μενίσκου]]","[εὐκλεής, μενίσκου]",2nd c. BC,"{'or': None, 'not_before': -200, 'not_after': ...",-200.0,-101.0,None,"[range, cent]",None,None


In [8]:
# trismegistos api
base_url = "https://www.trismegistos.org/dataservices/texrelations/{0}?source=phi"

In [9]:
%%time
resps = []
for n in PHI_sample["PHI_ID"][:100]:
    resp_json = requests.get(base_url.format(str(n))).json()
    resp_json = dict([(list(el.keys())[0], list(el.values())[0]) for el in resp_json])
    resps.append(resp_json)

CPU times: user 1.47 s, sys: 107 ms, total: 1.58 s
Wall time: 1min 1s


In [10]:
PHI_sample_alts = pd.DataFrame.from_records(resps)
PHI_sample_alts.head(10)

,TM_ID,EDB,EDH,EDCS,EDR,HE,UOXF,RIB,PHI,LUPA,...,TLA,ALA,CPI,GLAUx,Papyrus_Projekt,GEM,Innovating_knowledge,PUL,4CARE/DEChriM,NYU_Amheida
0,[907493],None,None,None,None,None,None,None,[180672],None,...,None,None,None,None,None,None,None,None,None,None
1,[884657],None,None,None,None,None,None,None,[293424],None,...,None,None,None,None,None,None,None,None,None,None
2,[782973],None,None,None,None,None,None,None,[344689],None,...,None,None,None,None,None,None,None,None,None,None
3,[868178],None,None,None,None,None,None,None,[282987],None,...,None,None,None,None,None,None,None,None,None,None
4,[865289],None,None,None,None,None,None,None,[170921],None,...,None,None,None,None,None,None,None,None,None,None
5,[276850],None,None,[39100774],[EDR115435],None,None,None,[187966],None,...,None,None,None,None,None,None,None,None,None,None
6,[934389],None,None,None,None,None,None,None,[181314],None,...,None,None,None,None,None,None,None,None,None,None
7,[762226],None,None,None,None,None,None,None,[170436],None,...,None,None,None,None,None,None,None,None,None,None
8,[764389],None,None,None,None,None,None,None,[169775],None,...,None,None,None,None,None,None,None,None,None,None
9,[813516],None,None,None,None,None,None,None,[147503],None,...,None,None,None,None,None,None,None,None,None,None


In [11]:
def get_inscription_data(phi_id):
    try:
        resp_json = requests.get(base_url.format(str(phi_id))).json()
        resp_json = dict([(list(el.keys())[0], list(el.values())[0]) for el in resp_json])
    except:
        time.sleep(1)
        try:
            resp_json = requests.get(base_url.format(str(phi_id))).json()
            resp_json = dict([(list(el.keys())[0], list(el.values())[0]) for el in resp_json])
        except:
            resp_json = {}
    return resp_json

In [12]:
%%time
### main run of the function

step=100
all_inscriptions = []
for num in range(0, 300, step):
    actual_nums = PHI[num:num+step]["PHI_ID"].tolist()
    with ThreadPoolExecutor(max_workers=step*1.5) as pool:
        currently_parsed = list(pool.map(get_inscription_data,actual_nums))
    all_inscriptions.extend(currently_parsed)

CPU times: user 5.51 s, sys: 539 ms, total: 6.05 s
Wall time: 5.06 s


In [17]:

pd.DataFrame(all_inscriptions)

,TM_ID,EDB,EDH,EDCS,EDR,HE,UOXF,RIB,PHI,LUPA,...,ALA,CPI,GLAUx,Papyrus_Projekt,GEM,Innovating_knowledge,PUL,4CARE/DEChriM,NYU_Amheida,WARNING
0,[786251],NaN,NaN,None,NaN,NaN,NaN,NaN,[1],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,[786252],NaN,NaN,None,NaN,NaN,NaN,NaN,[2],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,[786253],NaN,NaN,None,NaN,NaN,NaN,NaN,[3],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,[786254],NaN,NaN,None,NaN,NaN,NaN,NaN,[4],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,[786255],NaN,NaN,None,NaN,NaN,NaN,NaN,[5],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,[786544],NaN,NaN,None,NaN,NaN,NaN,NaN,[296],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
296,[786545],NaN,NaN,None,NaN,NaN,NaN,NaN,[297],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
297,[786546],NaN,NaN,None,NaN,NaN,NaN,NaN,[298],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
298,[786547],NaN,NaN,None,NaN,NaN,NaN,NaN,[299],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
list(range(0, len(PHI), step))[-1]

218100

In [26]:
checkspoints = list(range(0, len(PHI), 1000))[1:] + [list(range(0, len(PHI), step))[-1]] # its own range plus last element of steps
print(checkspoints)

[1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000, 11000, 12000, 13000, 14000, 15000, 16000, 17000, 18000, 19000, 20000, 21000, 22000, 23000, 24000, 25000, 26000, 27000, 28000, 29000, 30000, 31000, 32000, 33000, 34000, 35000, 36000, 37000, 38000, 39000, 40000, 41000, 42000, 43000, 44000, 45000, 46000, 47000, 48000, 49000, 50000, 51000, 52000, 53000, 54000, 55000, 56000, 57000, 58000, 59000, 60000, 61000, 62000, 63000, 64000, 65000, 66000, 67000, 68000, 69000, 70000, 71000, 72000, 73000, 74000, 75000, 76000, 77000, 78000, 79000, 80000, 81000, 82000, 83000, 84000, 85000, 86000, 87000, 88000, 89000, 90000, 91000, 92000, 93000, 94000, 95000, 96000, 97000, 98000, 99000, 100000, 101000, 102000, 103000, 104000, 105000, 106000, 107000, 108000, 109000, 110000, 111000, 112000, 113000, 114000, 115000, 116000, 117000, 118000, 119000, 120000, 121000, 122000, 123000, 124000, 125000, 126000, 127000, 128000, 129000, 130000, 131000, 132000, 133000, 134000, 135000, 136000, 137000, 138000, 139

In [30]:
%%time
### main run of the function

step=100
all_inscriptions = []
for num in range(0, 3001, step):
    actual_nums = PHI[num:num+step]["PHI_ID"].tolist()
    with ThreadPoolExecutor(max_workers=step*1.5) as pool:
        currently_parsed = list(pool.map(get_inscription_data,actual_nums))
    all_inscriptions.extend(currently_parsed)
    if num in checkspoints:
      s.write_file("SDAM_data/PHI/PHI_TM_{0}.csv".format(str(num)), pd.DataFrame(all_inscriptions))
      all_inscriptions = []

Your <class 'pandas.core.frame.DataFrame'> object has been succesfully written as "https://sciencedata.dk/files/SDAM_root/SDAM_data/PHI/PHI_TM_1000.csv"
Your <class 'pandas.core.frame.DataFrame'> object has been succesfully written as "https://sciencedata.dk/files/SDAM_root/SDAM_data/PHI/PHI_TM_2000.csv"
Your <class 'pandas.core.frame.DataFrame'> object has been succesfully written as "https://sciencedata.dk/files/SDAM_root/SDAM_data/PHI/PHI_TM_3000.csv"
CPU times: user 56.3 s, sys: 4.65 s, total: 1min
Wall time: 57.2 s


In [41]:
PHI_TM_1000 = s.read_file("SDAM_data/PHI/PHI_TM_{0}.csv".format(str(1000)))
PHI_TM_2000 = s.read_file("SDAM_data/PHI/PHI_TM_{0}.csv".format(str(2000)))
PHI_TM_3000 = s.read_file("SDAM_data/PHI/PHI_TM_{0}.csv".format(str(3000)))

In [50]:
PHI_TM_merge = pd.DataFrame()
for df in [PHI_TM_1000, PHI_TM_2000, PHI_TM_3000]:
  PHI_TM_merge = PHI_TM_merge.append(df)

In [60]:
[el for el in PHI_TM_merge["TM_ID"] if not isinstance(eval(el), list)]

[]

In [55]:
PHI_TM_merge[PHI_TM_merge["TM_ID"]]

KeyError: ignored

In [40]:
PHI_TM_1000.append(PHI_TM_2000)

,TM_ID,EDB,EDH,EDCS,EDR,HE,UOXF,RIB,PHI,LUPA,...,ALA,CPI,GLAUx,Papyrus_Projekt,GEM,Innovating_knowledge,PUL,4CARE/DEChriM,NYU_Amheida,WARNING
0,['786251'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,['1'],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,['786252'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,['2'],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,['786253'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,['3'],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,['786254'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,['4'],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,['786255'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,['5'],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,['788268'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,['2488'],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
996,['788269'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,['2489'],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
997,['788270'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,['2490'],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
998,['788271'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,['2491'],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
all_inscriptions

[{'TM_ID': ['786251'],
  'EDB': None,
  'EDH': None,
  'EDCS': None,
  'EDR': None,
  'HE': None,
  'UOXF': None,
  'RIB': None,
  'PHI': ['1'],
  'LUPA': None,
  'ISic': None,
  'IRT': None,
  'IGCyR': None,
  'Orient': None,
  'cisp': None,
  'DeM': None,
  'PSI': None,
  'PapNav-Duke': None,
  'Lexicological_lep': None,
  'Berliner_papyri': None,
  'Bremen_papyri': None,
  'chartes': None,
  'HispEpOl': None,
  'NaBuCCo': None,
  'Ashmolean_latin_inscriptions': None,
  'PUG': None,
  'IIP': None,
  'Dresden_papyri': None,
  'GVCYR': None,
  'Vindolanda': None,
  'Ghent_papyri': None,
  'Köln_papyri': None,
  'DDBDP': None,
  'HGV': None,
  'CACHKaRN': None,
  'PapNav-Stanford': None,
  'PapNav-Fordham': None,
  'PapNav-Berenike': None,
  'PapNav-Oslo': None,
  'PapNav-Toronto': None,
  'PapNav-Lund': None,
  'PapNav-Hermitage': None,
  'PapNav-Wisconsin': None,
  'PapNav-UTS': None,
  'PapNav-UPenn': None,
  'PSR': None,
  'PTS': None,
  'PapNav-Perkins': None,
  'PapNav-Petra': Non

In [ ]:
all_inscriptions_df = pd.DataFrame(all_inscriptions)
all_inscriptions_df.head(5)

In [ ]:
all_inscriptions_df.to_json("../data/large_data/TM_PHI.json")